## 원시 시장 데이터 정제
- 사용하지 않을 Feature 삭제
- 인구데이터 정제
- 결측치 보완 (대부분의 결측치는 자료조사를 통해 보완하였음)

In [13]:
import numpy as np 
import pandas as pd

In [30]:
# 데이터 불러오기
data = pd.read_csv('../data/전통시장데이터.csv', encoding = 'utf8')

### 사용하지 않을 Feature 삭제

In [31]:
# 사용하지 않는 Column 삭제
data.drop(['new index', '기존 index', '법정동코드', '행정동코드', '소재지지번주소', '읍면동명'], axis = 1, inplace = True)

In [32]:
# 행정동 인구 데이터
dong_ingu = data.iloc[:,24:45]
dong_ingu['시장명'] = data['시장명']

# 행정동 인구 삭제
dong_ingu_col = data.columns[24:45]
for col in dong_ingu_col :
  data.drop(col, axis = 1, inplace = True)

# 행정시구 인구 데이터
sigu_ingu = data.iloc[:,25:46]
sigu_ingu['시장명'] = data['시장명']

# 행정시구 인구 삭제
sigu_ingu_col = data.columns[25:46]
for col in sigu_ingu_col :
  data.drop(col, axis = 1, inplace = True)

### 인구데이터 정제

In [33]:
# "미성년자 = 0~19세 / 젊은청년 = 20~34세 / 소득인구 = 35~64세 / 노년인구 = 65세 이상"으로 정의
# 시구별, 동별 인구데이터 정제

sigu_ingu.drop(['시장명'], axis = 1, inplace = True)
sigu_ingu['시구 미성년자'] = sigu_ingu['행정시구 인구 0 - 4세'] + sigu_ingu['행정시구 인구 5 - 9세'] + sigu_ingu['행정시구 인구 10 - 14세'] + sigu_ingu['행정시구 인구 15 - 19세']
sigu_ingu['시구 젊은청년'] = sigu_ingu['행정시구 인구 20 - 24세'] + sigu_ingu['행정시구 인구 25 - 29세'] + sigu_ingu['행정시구 인구 30 - 34세']
sigu_ingu['시구 소득인구'] = sigu_ingu['행정시구 인구 35 - 39세'] + sigu_ingu['행정시구 인구 40 - 44세'] + sigu_ingu['행정시구 인구 45 - 49세'] + sigu_ingu['행정시구 인구 50 - 54세'] + \
                              sigu_ingu['행정시구 인구 55 - 59세'] + sigu_ingu['행정시구 인구 60 - 64세']
sigu_ingu['시구 노년인구'] =  sigu_ingu['행정시구 인구 65 - 69세'] + sigu_ingu['행정시구 인구 70 - 74세'] + sigu_ingu['행정시구 인구 75 - 79세'] + \
                              sigu_ingu['행정시구 인구 80 - 84세'] + sigu_ingu['행정시구 인구 85 - 89세'] +sigu_ingu['행정시구 인구 90 - 94세'] + sigu_ingu['행정시구 인구 95 - 99세'] + \
                              sigu_ingu['행정시구 인구 100+']
sigu_ingu = sigu_ingu.iloc[:,-4:]

dong_ingu.drop(['시장명'], axis = 1, inplace = True)
dong_ingu['동별 미성년자'] = dong_ingu['행정동 인구 0 - 4세'] + dong_ingu['행정동 인구 5 - 9세'] + dong_ingu['행정동 인구 10 - 14세'] + dong_ingu['행정동 인구 15 - 19세']
dong_ingu['동별 젊은청년'] = dong_ingu['행정동 인구 20 - 24세'] + dong_ingu['행정동 인구 25 - 29세'] + dong_ingu['행정동 인구 30 - 34세']
dong_ingu['동별 소득인구'] = dong_ingu['행정동 인구 35 - 39세'] + dong_ingu['행정동 인구 40 - 44세'] + dong_ingu['행정동 인구 45 - 49세'] + dong_ingu['행정동 인구 50 - 54세'] + \
                              dong_ingu['행정동 인구 55 - 59세'] + dong_ingu['행정동 인구 60 - 64세']
dong_ingu['동별 노년인구'] =  dong_ingu['행정동 인구 65 - 69세'] + dong_ingu['행정동 인구 70 - 74세'] + dong_ingu['행정동 인구 75 - 79세'] + \
                              dong_ingu['행정동 인구 80 - 84세'] + dong_ingu['행정동 인구 85 - 89세'] +dong_ingu['행정동 인구 90 - 94세'] + dong_ingu['행정동 인구 95 - 99세'] + \
                              dong_ingu['행정동 인구 100+']
dong_ingu = dong_ingu.iloc[:,-4:]   

new_df = pd.concat([data, sigu_ingu, dong_ingu], axis = 1)

### 결측치 보완

In [34]:
# DataFrame에서 Null값 체크  ->  "점포수, 공중화장실보유여부, 주차장보유여부, 운영기간"만 결측값 보유
null_check = new_df.isnull().sum()
print(null_check[null_check != 0])

점포수          12
공중화장실보유여부     4
주차장보유여부       5
운영기간         15
dtype: int64


In [35]:
# 총 1879개의 Data 중 29개의 Row에 결측치가 존재. Drop하여 결측치 처리
df = new_df.dropna()
df.reset_index(drop = True, inplace = True)
print('원시 데이터의 개수:', len(data))
print('결측치를 보유한 행의 개수:', len(new_df) - len(df))
print('최종 Data의 개수:', len(df))

원시 데이터의 개수: 1879
결측치를 보유한 행의 개수: 29
최종 Data의 개수: 1850


### 버스 정류장 데이터 합쳐주기

In [36]:
bus_data = pd.read_excel('../data/bus_with_market.xlsx')
df['버스정류장'] = bus_data['버스정류장']

<ipython-input-36-c5cf749392ff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['버스정류장'] = bus_data['버스정류장']


In [38]:
df.to_excel('../data/전통시장데이터_정리완료.xlsx')